# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [476]:
problem = '''The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits.'''

## 2. Generate the mathematical model

In [477]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [478]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for PRODA, S.A. Weekly Production

**Sets:**

*  $P = \{P1, P2, P3\}$: Set of products.

**Parameters:**

*  $s_p$: Smelting time per unit of product $p \in P$ at PRODA (in minutes).
*  $m_p$: Mechanization time per unit of product $p \in P$ (in minutes).
*  $a_p$: Assembly and packaging time per unit of product $p \in P$ (in minutes).
*  $c_{sp}$: Direct unit cost of smelting product $p \in P$ at PRODA.
*  $c_{ssp}$: Direct unit cost of subcontracted smelting for product $p \in \{P1, P2\}$.
*  $c_{mp}$: Direct unit cost of mechanization for product $p \in P$.
*  $c_{ap}$: Direct unit cost of assembly and packaging for product $p \in P$.
*  $pr_p$: Unit sales price for product $p \in P$.
*  $S$: Available smelting time at PRODA (in minutes).
*  $M$: Available mechanization time (in minutes).
*  $A$: Available assembly and packaging time (in minutes).

**Specific Parameter Values:**

| Parameter | P1 | P2 | P3 |
|---|---|---|---|
| $s_p$ | 6 | 10 | 8 |
| $m_p$ | 6 | 3 | 8 |
| $a_p$ | 3 | 2 | 2 |
| $c_{sp}$ | 0.30 | 0.50 | 0.40 |
| $c_{ssp}$ | 0.50 | 0.60 | - |
| $c_{mp}$ | 0.20 | 0.10 | 0.27 |
| $c_{ap}$ | 0.30 | 0.20 | 0.20 |
| $pr_p$ | 1.50 | 1.80 | 1.97 |

* $S = 8000$
* $M = 12000$
* $A = 10000$

**Decision Variables:**

*  $x_p$: Number of units of product $p \in P$ produced at PRODA (including smelting).
*  $y_p$: Number of units of product $p \in \{P1, P2\}$ with subcontracted smelting.

**Objective Function:**

Maximize weekly profit:

$$\sum_{p \in P} (pr_p - c_{sp} - c_{mp} - c_{ap}) x_p + \sum_{p \in \{P1, P2\}} (pr_p - c_{ssp} - c_{mp} - c_{ap}) y_p$$

**Constraints:**

* **Smelting Capacity:**
$$\sum_{p \in P} s_p x_p \leq S$$
* **Mechanization Capacity:**
$$\sum_{p \in P} m_p (x_p + y_p) \leq M$$
* **Assembly and Packaging Capacity:**
$$\sum_{p \in P} a_p (x_p + y_p) \leq A$$
* **Non-negativity:**
$$x_p \geq 0, \forall p \in P$$
$$y_p \geq 0, \forall p \in \{P1, P2\}$$

This model aims to determine the optimal production quantities for each product, considering both in-house and subcontracted smelting, while maximizing PRODA's weekly profit within the available production capacities. 


## 3. Generate the pyomo code

In [479]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [480]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define sets
products = ["P1", "P2", "P3"]
subcontractable_products = ["P1", "P2"]

# Define parameters
smelting_time = {"P1": 6, "P2": 10, "P3": 8}  
mechanization_time = {"P1": 6, "P2": 3, "P3": 8} 
assembly_time = {"P1": 3, "P2": 2, "P3": 2}  

# Cost parameters
smelting_cost = {"P1": 0.30, "P2": 0.50, "P3": 0.40}
subcontracted_smelting_cost = {"P1": 0.50, "P2": 0.60}
mechanization_cost = {"P1": 0.20, "P2": 0.10, "P3": 0.27}
assembly_cost = {"P1": 0.30, "P2": 0.20, "P3": 0.20}

# Selling price
selling_price = {"P1": 1.50, "P2": 1.80, "P3": 1.97}

# Available production time
available_smelting_time = 8000
available_mechanization_time = 12000
available_assembly_time = 10000

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(products, domain=pyo.NonNegativeReals) # In-house production
model.y = pyo.Var(subcontractable_products, domain=pyo.NonNegativeReals) # Subcontracted production

# Define objective function
model.profit = pyo.Objective(expr = 
    sum((selling_price[p] - smelting_cost[p] - mechanization_cost[p] - assembly_cost[p]) * model.x[p] for p in products) +
    sum((selling_price[p] - subcontracted_smelting_cost[p] - mechanization_cost[p] - assembly_cost[p]) * model.y[p] for p in subcontractable_products),
    sense=pyo.maximize)

# Define constraints
model.smelting_capacity = pyo.Constraint(expr = sum(smelting_time[p] * model.x[p] for p in products) <= available_smelting_time)

model.mechanization_capacity = pyo.Constraint(expr = 
    sum(mechanization_time[p] * (model.x[p] + model.y[p]) for p in subcontractable_products) + 
    sum(mechanization_time[p] * model.x[p] for p in products if p not in subcontractable_products) 
    <= available_mechanization_time)

model.assembly_capacity = pyo.Constraint(expr = 
    sum(assembly_time[p] * (model.x[p] + model.y[p]) for p in subcontractable_products) +
    sum(assembly_time[p] * model.x[p] for p in products if p not in subcontractable_products) 
    <= available_assembly_time)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print results
print("Optimal Production Plan:")
for p in products:
    print(f"Product {p}: {model.x[p].value:.2f} (in-house)")
for p in subcontractable_products:
    print(f"Product {p}: {model.y[p].value:.2f} (subcontracted)")

print(f"\nTotal Profit: ${model.profit():.2f}")
```

**Explanation of Sample Data:**

The provided code directly uses the specific parameter values given in the problem statement as sample data. For instance:

* `smelting_time = {"P1": 6, "P2": 10, "P3": 8}` represents the $s_p$ parameters.
* `available_smelting_time = 8000` corresponds to the $S$ parameter.
* The same applies to other parameters like `mechanization_time`, `assembly_time`, `smelting_cost`, etc. 

This code first sets up the model by defining sets, parameters, and decision variables based on the provided data. Then, it formulates the objective function to maximize profit and includes constraints for production capacities. Finally, it calls a solver to find the optimal production plan and displays the results. 


## 4. Input problem data and try running the generated code

In [481]:
import pyomo.environ as pyo

# Define sets
products = ["P1", "P2", "P3"]
subcontractable_products = ["P1", "P2"]

# Define parameters
smelting_time = {"P1": 6, "P2": 10, "P3": 8}  
mechanization_time = {"P1": 6, "P2": 3, "P3": 8} 
assembly_time = {"P1": 3, "P2": 2, "P3": 2}  

# Cost parameters
smelting_cost = {"P1": 0.30, "P2": 0.50, "P3": 0.40}
subcontracted_smelting_cost = {"P1": 0.50, "P2": 0.60}
mechanization_cost = {"P1": 0.20, "P2": 0.10, "P3": 0.27}
assembly_cost = {"P1": 0.30, "P2": 0.20, "P3": 0.20}

# Selling price
selling_price = {"P1": 1.50, "P2": 1.80, "P3": 1.97}

# Available production time
available_smelting_time = 8000
available_mechanization_time = 12000
available_assembly_time = 10000

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(products, domain=pyo.NonNegativeReals) # In-house production
model.y = pyo.Var(subcontractable_products, domain=pyo.NonNegativeReals) # Subcontracted production

# Define objective function
model.profit = pyo.Objective(expr = 
    sum((selling_price[p] - smelting_cost[p] - mechanization_cost[p] - assembly_cost[p]) * model.x[p] for p in products) +
    sum((selling_price[p] - subcontracted_smelting_cost[p] - mechanization_cost[p] - assembly_cost[p]) * model.y[p] for p in subcontractable_products),
    sense=pyo.maximize)

# Define constraints
model.smelting_capacity = pyo.Constraint(expr = sum(smelting_time[p] * model.x[p] for p in products) <= available_smelting_time)

model.mechanization_capacity = pyo.Constraint(expr = 
    sum(mechanization_time[p] * (model.x[p] + model.y[p]) for p in subcontractable_products) + 
    sum(mechanization_time[p] * model.x[p] for p in products if p not in subcontractable_products) 
    <= available_mechanization_time)

model.assembly_capacity = pyo.Constraint(expr = 
    sum(assembly_time[p] * (model.x[p] + model.y[p]) for p in subcontractable_products) +
    sum(assembly_time[p] * model.x[p] for p in products if p not in subcontractable_products) 
    <= available_assembly_time)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print results
print("Optimal Production Plan:")
for p in products:
    print(f"Product {p}: {model.x[p].value:.2f} (in-house)")
for p in subcontractable_products:
    print(f"Product {p}: {model.y[p].value:.2f} (subcontracted)")

print(f"\nTotal Profit: ${model.profit():.2f}")

Optimal Production Plan:
Product P1: 0.00 (in-house)
Product P2: 800.00 (in-house)
Product P3: 0.00 (in-house)
Product P1: 0.00 (subcontracted)
Product P2: 3200.00 (subcontracted)

Total Profit: $3680.00


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [474]:
print(response.text)

## Mathematical Optimization Model for PRODA, S.A. Weekly Production

**Sets:**

* $P = \{P1, P2, P3\}$: Set of products.

**Parameters:**

* **Production Costs:**
    * $c_{i}^{s}$: Unit cost of smelting product $i$ at PRODA, $\forall i \in P$.
    * $c_{i}^{ss}$: Unit cost of subcontracted smelting for product $i$, $\forall i \in \{P1, P2\}$.
    * $c_{i}^{m}$: Unit cost of mechanisation for product $i$, $\forall i \in P$.
    * $c_{i}^{ap}$: Unit cost of assembly and packaging for product $i$, $\forall i \in P$.
* **Sales Price:**
    * $p_{i}$: Unit sales price for product $i$, $\forall i \in P$.
* **Production Times:**
    * $t_{i}^{s}$: Time (in minutes) required for smelting one unit of product $i$ at PRODA, $\forall i \in P$.
    * $t_{i}^{m}$: Time (in minutes) required for mechanization of one unit of product $i$, $\forall i \in P$.
    * $t_{i}^{ap}$: Time (in minutes) required for assembly and packaging of one unit of product $i$, $\forall i \in P$.
* **Capacity:**
    * $C

In [475]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define sets
P = ['P1', 'P2', 'P3']

# Define parameters (Sample Data)
c_s = {'P1': 0.30, 'P2': 0.50, 'P3': 0.40}
c_ss = {'P1': 0.50, 'P2': 0.60}
c_m = {'P1': 0.20, 'P2': 0.10, 'P3': 0.27}
c_ap = {'P1': 0.30, 'P2': 0.20, 'P3': 0.20}
p = {'P1': 1.50, 'P2': 1.80, 'P3': 1.97}
t_s = {'P1': 6, 'P2': 10, 'P3': 8}
t_m = {'P1': 6, 'P2': 3, 'P3': 8}
t_ap = {'P1': 3, 'P2': 2, 'P3': 2}
C_s = 8000
C_m = 12000
C_ap = 10000

# Create model
model = pyo.ConcreteModel()

# Define variables
model.x = pyo.Var(P, nonneg=True) # units produced at PRODA
model.y = pyo.Var(['P1', 'P2'], nonneg=True) # units subcontracted

# Define objective function
model.profit = pyo.Objective(
    expr = sum(p[i] * (model.x[i] + model.y[i]) for i in P) - 
           sum((c_s[i] + c_m[i] + c_ap[i]) * model.x[i] for i in P) -
           sum(c_ss[i] * model.y[i] for i in ['P1', 'P2']),
    sense=pyo.maximize
)

# Define constraints
model.smelting_capacity = pyo.Constraint(expr = sum(t_s[